In [1]:
# parameters
txt_path = "pairs.txt"
single_pic = "object_images"
combin_folder = "Pictures"

In [2]:
from resmem.model import ResMem, transformer

# load model
model = ResMem(pretrained=True)
model.eval() # Set the model to inference mode.

/project/wilma/katyzhang/resmem/resmem/../resmem/model.pt


ResMem(
  (features): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [ ]:
import os

# load pairs
with open(txt_path) as f:
    pairs = f.readlines()
# Loop: calcualte each photo's memorability
picture_paths = []
for pair in pairs:
    concept_1 = pair.split(", ")[0].replace(" ","_")
    concept_2 = pair.split(", ")[1][:-1].replace(" ","_")
    combin_path = os.path.join(combin_folder,concept_1+"+"+concept_2)
    # Loop: within each folder, calculate each photo's memorability
    for filename in os.listdir(combin_path):
        path_tmp = []
        path_tmp.append(os.path.join(single_pic,concept_1, filename.split("&&")[0]+".jpg"))
        path_tmp.append(os.path.join(single_pic,concept_2, filename.split("&&")[1][:-11]+".jpg"))
        path_tmp.append(os.path.join(combin_path, filename))
        picture_paths.append(path_tmp)

In [ ]:
picture_paths

[['object_images/grapefruit/grapefruit_10s.jpg',
  'object_images/granola/granola_07s.jpg',
  'Pictures/grapefruit+granola/grapefruit_10s&&granola_07s_00001_.png'],
 ['object_images/grapefruit/grapefruit_14s.jpg',
  'object_images/granola/granola_04s.jpg',
  'Pictures/grapefruit+granola/grapefruit_14s&&granola_04s_00001_.png'],
 ['object_images/grapefruit/grapefruit_15s.jpg',
  'object_images/granola/granola_14s.jpg',
  'Pictures/grapefruit+granola/grapefruit_15s&&granola_14s_00001_.png'],
 ['object_images/grapefruit/grapefruit_02s.jpg',
  'object_images/granola/granola_02s.jpg',
  'Pictures/grapefruit+granola/grapefruit_02s&&granola_02s_00001_.png'],
 ['object_images/grapefruit/grapefruit_18n.jpg',
  'object_images/granola/granola_11s.jpg',
  'Pictures/grapefruit+granola/grapefruit_18n&&granola_11s_00001_.png'],
 ['object_images/grapefruit/grapefruit_16n.jpg',
  'object_images/granola/granola_15s.jpg',
  'Pictures/grapefruit+granola/grapefruit_16n&&granola_15s_00001_.png'],
 ['object_

# Calculate memorability

In [5]:
from PIL import Image
import json

result_path = txt_path.split('.txt')[0] + '_memorability.json'

# Loop: calcualte each photo's memorability
memorabilty_lst = []
for pair_path in picture_paths:
    memorability_tmp = {}
    for index, pic_path in enumerate(pair_path):
        img = Image.open(pic_path) # loads image into memory
        img = img.convert('RGB') # convert image into RGB, for instance if it's a PNG (RGBA) or if it's black and white.
        image_x = transformer(img) # Run the preprocessing function
        prediction = model(image_x.view(-1, 3, 227, 227)) # For a single image, the image must be reshaped into a batch with size 1.

        concept = pic_path.split("/")[1]
        memorability_tmp[concept] = prediction.item()
        img.close()
    memorabilty_lst.append(memorability_tmp)

with open(result_path, 'w') as f:
    json.dump(memorabilty_lst, f, indent=4)


In [6]:
from PIL import Image
import json

result_path = txt_path.split('.txt')[0] + '_memorability_fullname.json'

# Loop: calcualte each photo's memorability
memorabilty_lst = []
for pair_path in picture_paths:
    memorability_tmp = {}
    for index, pic_path in enumerate(pair_path):
        img = Image.open(pic_path) # loads image into memory
        img = img.convert('RGB') # convert image into RGB, for instance if it's a PNG (RGBA) or if it's black and white.
        image_x = transformer(img) # Run the preprocessing function
        prediction = model(image_x.view(-1, 3, 227, 227)) # For a single image, the image must be reshaped into a batch with size 1.

        concept = pic_path.split("/")[2][:-4]
        memorability_tmp[concept] = prediction.item()
        img.close()
    memorabilty_lst.append(memorability_tmp)

with open(result_path,'w') as f:
    json.dump(memorabilty_lst, f, indent=4)